In [ ]:
#load the necessary libraries
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.datasets import cifar10
from keras.utils import to_categorical

In [ ]:
#load the dataset and reshape its dimensons to a size which led to the best results in past experiments (150x150)

#load the training set (about 80% of the entire dataset)

train_datagen = ImageDataGenerator(rescale=1./255)
X_train = train_datagen.flow_from_directory(
        '.../TIROCINIO/train_okco',
        target_size=(150,150),
        batch_size=50,
        shuffle=False,
        class_mode="categorical"
    )

Found 2407 images belonging to 7 classes.


In [ ]:
#load the test set (about 20% of the entire dataset). Images are different from ones of the training set

test_datagen = ImageDataGenerator(rescale=1./255)
X_test = train_datagen.flow_from_directory(
        '.../TIROCINIO/test_okc',
        target_size=(150,150),
        batch_size=50,
        shuffle=False,
        class_mode="categorical"
    )

Found 581 images belonging to 7 classes.


In [ ]:
#load the VGG16 model from Keras, without the top layers

base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
base_model.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [ ]:
#LAYERS FOR FEATURE EXTRACTION

flat1 = Flatten()(base_model.layers[-1].output)

# define new model
model = Model(inputs=base_model.inputs, outputs=flat1)

In [ ]:
#LAYERS FOR CLASSIFICATION

# add new classifier layers

flat1 = Flatten()(base_model.layers[-1].output)
class1 = Dense(4096, activation='relu')(flat1)
output = Dense(7, activation='softmax')(class1)
# define new model
model = Model(inputs=base_model.inputs, outputs=output)

In [ ]:
#Set all layers,except the last one to not trainable

for layer in model.layers[:-1]: layer.trainable=False

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [ ]:
#compile the model

from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train the model

from sklearn.metrics import confusion_matrix
epochs=20
for i in range(epochs):

 model.fit(
  X_train,
  validation_data=X_test,
  epochs=1
 )
 

49/49 [==============================] - 656s 13s/step - loss: 0.4202 - accuracy: 0.8625 - val_loss: 1.0946 - val_accuracy: 0.6592


In [ ]:
#confusion matrix and classification report calculated for training set

import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
batch_size = X_test.batch_size
num_of_test_samples = X_test.samples
predictions = model.predict(X_test,  num_of_test_samples // batch_size+1)

y_pred2 = np.argmax(predictions, axis=1)

true_classes = X_test.classes
class_labels = list(X_test.class_indices.keys())   

print(class_labels)

print(confusion_matrix(X_test.classes, y_pred2))

report = classification_report(true_classes, y_pred2, target_names=class_labels)
print(report)

['1_anger', '2_contempt', '3_disgust', '4_fear', '5_happy', '6_sadness', '7_surprise']
[[ 38  16   9   0   4   9   0]
 [  5  16   0   0  11   0   0]
 [  6   0  63   0  14   1   9]
 [  0   0   0   0  44   0  11]
 [  2   1   0   0 122   3   6]
 [ 13   0   2   0  14  17   7]
 [  0   6   0   0   4   1 127]]
              precision    recall  f1-score   support

     1_anger       0.59      0.50      0.54        76
  2_contempt       0.41      0.50      0.45        32
   3_disgust       0.85      0.68      0.75        93
      4_fear       0.00      0.00      0.00        55
     5_happy       0.57      0.91      0.70       134
   6_sadness       0.55      0.32      0.40        53
  7_surprise       0.79      0.92      0.85       138

    accuracy                           0.66       581
   macro avg       0.54      0.55      0.53       581
weighted avg       0.61      0.66      0.62       581



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#confusion matrix and classification report calculated for test set

import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
batch_size = X_train.batch_size
num_of_test_samples = X_train.samples
predictions = model.predict(X_train)

y_pred = np.argmax(predictions, axis=1)

true_classes = X_train.classes
class_labels = list(X_train.class_indices.keys())   

print(class_labels)

print(confusion_matrix(X_train.classes, y_pred))

report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

['1_anger', '2_contempt', '3_disgust', '4_fear', '5_happy', '6_sadness', '7_surprise']
[[388  30   0   0  13   9   5]
 [  0  85   0   0   0   0   0]
 [  5   2 269   0  34  13  29]
 [  1  31   1  85  49  25  24]
 [  0   3   0   0 524   0   8]
 [  0   0   1   0   2 221   0]
 [  0   3   0   0   3   1 543]]
              precision    recall  f1-score   support

     1_anger       0.98      0.87      0.92       445
  2_contempt       0.55      1.00      0.71        85
   3_disgust       0.99      0.76      0.86       352
      4_fear       1.00      0.39      0.56       216
     5_happy       0.84      0.98      0.90       535
   6_sadness       0.82      0.99      0.90       224
  7_surprise       0.89      0.99      0.94       550

    accuracy                           0.88      2407
   macro avg       0.87      0.85      0.83      2407
weighted avg       0.90      0.88      0.87      2407

